In [ ]:
import pandas as pd
import numpy as np
from matminer.datasets.dataset_retrieval import load_dataset
from modnet.featurizers.presets import Matminer2023Featurizer
from modnet.preprocessing import nmi_target ,get_cross_nmi, get_features_relevance_redundancy, get_features_dyn
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_matminer_phonon = load_dataset('matbench_phonons')

In [ ]:
df_matminer_ph_dielec = load_dataset('phonon_dielectric_mp')
df_matminer_ph_dielec.set_index('mpid',inplace=True)

In [ ]:
for row, col in df_matminer_ph_dielec.iterrows():
    for row2,col2 in df_matminer_phonon.iterrows():
        if col['structure'] == df_matminer_phonon.loc[row2,'structure']:
            df_matminer_phonon.loc[row2, 'mpid']= row
df = df_matminer_phonon.drop(columns=['last phdos peak'])
df.set_index('mpid',inplace=True)

In [ ]:
lobpy_stat= pd.read_csv('Small_basis_sum_stat.csv', sep=';', index_col='mp-ids')
lobpy_stat.sort_index(inplace=True)

In [ ]:
feat = Matminer2023Featurizer()

In [ ]:
df_comp = feat.featurize_composition(df)

In [ ]:
df_comp[df_comp.columns[df_comp.isnull().any()]].isnull().sum()

In [ ]:
df_st = feat.featurize_structure(df)

In [ ]:
df_merge = pd.concat([df_st, df_comp], axis=1)

In [ ]:
# find columns with max null values
df_merge[df_merge.columns[df_merge.isnull().any()]].isnull().sum()

In [ ]:
# drop columns with maximim null values
df_fil = df_merge.drop(columns=['GlobalSymmetryFeatures|crystal_system',
                      'AtomicPackingEfficiency|mean simul. packing efficiency',
                      'AtomicPackingEfficiency|mean abs simul. packing efficiency',
                      'Miedema|Miedema_deltaH_inter',
                      'Miedema|Miedema_deltaH_amor',
                      'Miedema|Miedema_deltaH_ss_min',
                      'YangSolidSolution|Yang delta'])

In [ ]:
df_fil.dropna(inplace=True)

In [ ]:
for feat in lobpy_stat.columns:
    for row, col in df_fil.iterrows():
        #print(row)
        df_fil.loc[row,feat] = lobpy_stat.loc[row,feat]
    

In [ ]:
for row, col in df_fil.iterrows():
    df_fil.loc[row,'last_phdos_peak']=df_matminer_ph_dielec.loc[row,'last phdos peak']

In [ ]:
df_fil.to_pickle('dataforml.pkl')